In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


In [3]:
pd.__version__

'1.4.2'

In [4]:
sklearn.__version__

'1.4.2'

In [5]:
!pip install pyarrow

In [6]:
df_jan_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [7]:
df_jan_2023.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [8]:
df_feb_2023.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [9]:
num_columns = df_jan_2023.shape[1]
print(f'The January data contains {num_columns} columns.')

The January data contains 19 columns.


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [10]:
duration_diff_sr = df_jan_2023.tpep_dropoff_datetime - df_jan_2023.tpep_pickup_datetime
df_jan_2023['duration'] = duration_diff_sr.apply(lambda x: x.total_seconds()/60)

In [11]:
print(f'''The standard deviation in minutes of the ride duration is {np.round(np.std(df_jan_2023['duration']), 2)}.''')


The standard deviation in minutes of the ride duration is 42.59.


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [11]:
duration_df_without_outliers = df_jan_2023['duration'][(df_jan_2023['duration'] >= 1) & (df_jan_2023['duration'] <= 60)]
duration_df_with_outliers = df_jan_2023['duration']



In [16]:
fraction_left = 100 * duration_df_without_outliers.shape[0] / duration_df_with_outliers.shape[0]
print(f'The fraction of records left after dropping the outliers is {fraction_left:.0f}%.')


The fraction of records left after dropping the outliers is 98%.


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?


* 2
* 155
* 345
* 515
* 715

In [17]:
jan_wo_df = df_jan_2023[(df_jan_2023['duration'] >= 1) & (df_jan_2023['duration'] <= 60)]
jan_wo_pu_do_loc_df = jan_wo_df[['PULocationID', 'DOLocationID']].astype(str)


In [18]:
jan_wo_pu_do_loc_dict = jan_wo_pu_do_loc_df.to_dict(orient='records')

In [19]:
dv = DictVectorizer()


In [20]:
jan_wo_transf_df = dv.fit_transform(jan_wo_pu_do_loc_dict)
print(f'The feature matrix has {jan_wo_transf_df.shape[1]} columns.')

The feature matrix has 515 columns.




## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?


* 3.64
* 7.64
* 11.64
* 16.64

In [21]:
x_jan = jan_wo_transf_df.copy()
y_jan = jan_wo_df['duration'].copy()

In [22]:
lr_jar = LinearRegression()
lr_jar.fit(x_jan, y_jan)

LinearRegression()

In [23]:
y_jan_pred = lr_jar.predict(x_jan)
print(f'The RMSE on the January data is: {round(root_mean_squared_error(y_jan, y_jan_pred), 2)}.')

The RMSE on the January data is: 7.65.


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [24]:
feb_duration_diff_sr = df_feb_2023.tpep_dropoff_datetime - df_feb_2023.tpep_pickup_datetime
df_feb_2023['duration'] = feb_duration_diff_sr.apply(lambda x: x.total_seconds()/60)

In [25]:
feb_wo_df = df_feb_2023[(df_feb_2023['duration'] >= 1) & (df_feb_2023['duration'] <= 60)]
feb_wo_pu_do_loc_df = feb_wo_df[['PULocationID', 'DOLocationID']].astype(str)

In [26]:
feb_wo_pu_do_loc_dict = feb_wo_pu_do_loc_df.to_dict(orient='records')


In [ ]:
feb_wo_transf_df = dv.fit_transform(feb_wo_pu_do_loc_dict)


In [1]:
x_feb = feb_wo_transf_df.copy()
y_feb = feb_wo_df['duration'].copy()

NameError: name 'feb_wo_transf_df' is not defined

In [28]:
lr_feb = LinearRegression()
lr_feb.fit(x_feb, y_feb)

LinearRegression()

In [32]:
y_feb_pred = lr_feb.predict(x_feb)

print(f"The RMSE for the February data is: {root_mean_squared_error(y_feb, y_feb_pred):.2f}.")

The RMSE for the February data is: 7.78.
